In [ ]:
!pip install psycopg2-binary
try:
    from faker import Faker
except:
   !pip install faker 
   from faker import Faker
    
try:
    import psycopg2 
except:
    !pip install psycopg2-binary 
    import psycopg2
    
try:
    from sqlalchemy import create_engine
except:
    !pip install sqlalchemy
    from sqlalchemy import create_engine
    
    
try:
    import pandas as pd 
except:
    !pip install pandas
    import pandas as pd 
     

#Connect to Postgress Database and inspect table names

In [2]:
from sqlalchemy import inspect,create_engine
import psycopg2

host="postgres_storage"
database="csv_db"
user="aawadallah"
password="1234"
port='5432'
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')
insp = inspect(engine)
print(insp.get_table_names())

['users2021']


#make Fake data using Faker, and save it to Data.csv 

In [3]:
from faker import Faker
import csv
import pandas as pd 
output=open('/home/sharedVol/data.csv','w')
fake=Faker()
header=['name','age','street','city','state','zip','lng','lat']
mywriter=csv.writer(output)
mywriter.writerow(header)
for r in range(100):
    row =[fake.name(),fake.random_int(min=18,max=80, step=1), 
                       fake.street_address(), fake.city(),fake.state(),
                       fake.zipcode(),fake.longitude(),fake.latitude()]
    mywriter.writerow(row)

output.close()



#Read CSV file using pandas and convert it to dataframe

In [4]:
DF=pd.read_csv('/home/sharedVol/data.csv')

DF.head(10)

,name,age,street,city,state,zip,lng,lat
0,Joseph Howard,67,5112 Stephanie Isle,West Carol,Colorado,55298,-171.999358,-3.741826
1,Sarah Gray,22,6088 Jason Lane Apt. 152,East Brandon,California,52939,172.076006,65.996260
2,Kelly Morton,22,3596 Parks Villages,North Johnmouth,Nebraska,72568,7.983681,33.404717
3,Nicole Hughes,37,80331 Martin Island,West Daniellechester,Minnesota,34393,69.406849,61.139549
4,Beverly Lane,49,43198 Marks Mountains Apt. 633,Maymouth,Idaho,94539,-65.149032,-25.641057
5,Michael Wood,79,76232 Cortez Ridge Apt. 646,Spencerberg,Michigan,29033,-38.132979,75.382036
6,Calvin Erickson,37,83181 Brooks Stream Suite 069,North Heathermouth,Connecticut,25081,85.851657,80.332328
7,Tony Espinoza,20,171 Alexander Park Apt. 638,South Jessica,Georgia,57475,69.560803,81.899154
8,Paul Fields,41,4937 Cheryl Valley,New Robert,New Mexico,38210,61.004976,14.052493
9,Ann Whitehead,48,00671 Isabella Run,Rodriguezland,Delaware,84307,-134.066466,52.381862


#Push Dataframe to postgress database 

In [5]:
DF.to_sql('users2021', engine,if_exists='replace',index=False)

In [11]:
#verfiy data by reading it from sql.

In [6]:
DF2=pd.read_sql("SELECT * FROM users2021" , engine);
DF2.head(10)

,name,age,street,city,state,zip,lng,lat
0,Joseph Howard,67,5112 Stephanie Isle,West Carol,Colorado,55298,-171.999358,-3.741826
1,Sarah Gray,22,6088 Jason Lane Apt. 152,East Brandon,California,52939,172.076006,65.996260
2,Kelly Morton,22,3596 Parks Villages,North Johnmouth,Nebraska,72568,7.983681,33.404717
3,Nicole Hughes,37,80331 Martin Island,West Daniellechester,Minnesota,34393,69.406849,61.139549
4,Beverly Lane,49,43198 Marks Mountains Apt. 633,Maymouth,Idaho,94539,-65.149032,-25.641057
5,Michael Wood,79,76232 Cortez Ridge Apt. 646,Spencerberg,Michigan,29033,-38.132979,75.382036
6,Calvin Erickson,37,83181 Brooks Stream Suite 069,North Heathermouth,Connecticut,25081,85.851657,80.332328
7,Tony Espinoza,20,171 Alexander Park Apt. 638,South Jessica,Georgia,57475,69.560803,81.899154
8,Paul Fields,41,4937 Cheryl Valley,New Robert,New Mexico,38210,61.004976,14.052493
9,Ann Whitehead,48,00671 Isabella Run,Rodriguezland,Delaware,84307,-134.066466,52.381862


In [7]:
DF2.to_csv("/home/sharedVol/data2.csv")

#Connect to Mongo 

In [24]:
from pymongo import MongoClient
client = MongoClient('mongo:27017', 
                        username='aawadallah',
                         password='1234')

In [ ]:
#Create Database

In [25]:
db = client['users2021']

In [ ]:
#Create Collection 

In [26]:
collection = db['users']

Pust Dataframe to mongo 

In [27]:
DF2.reset_index(inplace=True)
data_dict = DF2.to_dict("records")
# Insert collection
collection.insert_many(data_dict)

In [31]:
collection.estimated_document_count()

100

In [ ]:
#Read Documents from mongo 

In [16]:
cursor = collection.find({})
for document in cursor:
    print(document)

#AIRFLOW DAG

In [ ]:
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime
import subprocess
from sqlalchemy import create_engine
from pymongo import MongoClient
import pandas as pd

host = "postgres_storage"
database = "csv_data"
user = "aawadallah"
password = "1234"
port = '5432'
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')

client = MongoClient('mongo:27017',
                     username='aawadallah',
                     password='1234')
mongodb = client['users2021']
collection = mongodb['users']


def _read_table_as_DF():
    DF = pd.read_sql("SELECT * FROM users2021;", engine)
    DF.to_csv("/home/sharedVol/data2.csv")
    print(DF.head(5))


def _push_DF_to_Mongo():
    DF2 = pd.read_csv("/home/sharedVol/data2.csv")
    DF2.reset_index(inplace=True)
    data_dict = DF2.to_dict("records")
    # Insert collection
    collection.insert_many(data_dict)


def _read_from_MongoDB():
    print('number of documents in mongoDB = ', collection.estimated_document_count());


def _install_tools():
    try:
        from faker import Faker
    except:
        subprocess.check_call(['pip', 'install', 'faker'])
        from faker import Faker

    try:
        import psycopg2
    except:
        subprocess.check_call(['pip', 'install', 'psycopg2-binary'])
        import psycopg2

    try:
        from sqlalchemy import create_engine
    except:
        subprocess.check_call(['pip', 'install', 'sqlalchemy'])
        from sqlalchemy import create_engine

    try:
        from pymongo import MongoClient
    except:
        subprocess.check_call(['pip', 'install', 'pymongo'])
        from pymongo import MongoClient

    try:
        import pandas as pd
    except:
        subprocess.check_call(['pip', 'install', 'pandas'])
        import pandas as pd


with DAG("etl_postgresql2mongo", start_date=datetime(2021, 1, 1),
         schedule_interval="*/5 * * * *", catchup=False) as dag:
    install_tools = PythonOperator(
        task_id="install_tools",
        python_callable=_install_tools
    )
    read_table_as_DF = PythonOperator(
        task_id="read_table_as_DF",
        python_callable=_read_table_as_DF
    )

    push_DF_to_Mongo = PythonOperataor(
        task_id="push_DF_to_Mongo",
        python_callable=_push_DF_to_Mongo
    )

    read_from_MongoDB = PythonOperator(
        task_id="read_from_MongoDB",
        python_callable=_read_from_MongoDB
    )

    install_tools >> read_table_as_DF >> push_DF_to_Mongo >> read_from_MongoDB